## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cape Town,ZA,-33.9258,18.4232,66.11,37,0,5.75,clear sky
1,1,Port Alfred,ZA,-33.5906,26.8910,61.18,63,87,4.00,overcast clouds
2,2,Barrow,US,71.2906,-156.7887,32.52,98,100,16.28,snow
3,3,Rikitea,PF,-23.1203,-134.9692,73.22,74,1,21.16,clear sky
4,4,Ilulissat,GL,69.2167,-51.1000,43.57,80,100,6.58,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                      (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Rikitea,PF,-23.1203,-134.9692,73.22,74,1,21.16,clear sky
6,6,Mahebourg,MU,-20.4081,57.7000,73.69,60,40,19.57,scattered clouds
8,8,Vaini,TO,-21.2000,-175.2000,71.76,78,40,10.36,scattered clouds
9,9,Karaton,ID,-6.3424,106.1045,74.80,93,29,0.74,scattered clouds
12,12,Atuona,PF,-9.8000,-139.0333,77.00,68,2,21.52,clear sky


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

Unnamed: 0     409
City           409
Country        406
Lat            409
Lng            409
Max Temp       409
Humidity       409
Cloudiness     409
Wind Speed     409
Description    409
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0     406
City           406
Country        406
Lat            406
Lng            406
Max Temp       406
Humidity       406
Cloudiness     406
Wind Speed     406
Description    406
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Rikitea,PF,73.22,clear sky,-23.1203,-134.9692,
6,Mahebourg,MU,73.69,scattered clouds,-20.4081,57.7000,
8,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,
9,Karaton,ID,74.80,scattered clouds,-6.3424,106.1045,
12,Atuona,PF,77.00,clear sky,-9.8000,-139.0333,
13,Faanui,PF,78.51,scattered clouds,-16.4833,-151.7500,
20,Hasaki,JP,81.28,overcast clouds,35.7333,140.8333,
21,Lorengau,PG,79.79,overcast clouds,-2.0226,147.2712,
26,Kapaa,US,75.18,clear sky,22.0752,-159.3190,
28,Bathsheba,BB,83.93,scattered clouds,13.2167,-59.5167,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = (f'{lat},{lng}')
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # 6e. Make request and retrieve the JSON data from the search. 
   
        hotels = requests.get(base_url,params=params).json()
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]

    
    except (IndexError):
        print("Hotel not found and skipping")

Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping
Hotel not found and skipping


In [19]:
import numpy as np
hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
hotel_df.count()

City           406
Country        406
Max Temp       406
Description    406
Lat            406
Lng            406
Hotel Name     382
dtype: int64

In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df = hotel_df.dropna()
hotel_clean_df.count()

City           382
Country        382
Max Temp       382
Description    382
Lat            382
Lng            382
Hotel Name     382
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name} °F</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp}°F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))